In [ ]:
#from https://towardsdatascience.com/the-lesser-known-rising-application-of-llms-775834116477

!git clone https://github.com/shawngraham/structured-cultural-heritage-crime

In [3]:
%cd structured-cultural-heritage-crime

/content/structured-cultural-heritage-crime


In [ ]:
!pip install -r requirements.txt
#should remove mistral and replace with openai


In [6]:
!pip install python-dotenv

In [ ]:
!pip install langchain-openai


In [6]:
import os
import json
import time
from pathlib import Path

import pandas as pd
from langchain.output_parsers import PydanticOutputParser
#from langchain_mistralai.chat_models import ChatMistralAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

from core import run
from prompt import DEFAULT_BASE_PROMPT, create_prompt
from schemas import CulturalHeritageSchema


In [7]:

from google.colab import userdata
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [9]:
!unzip ../texts.zip

Archive:  ../texts.zip
   creating: texts/
  inflating: texts/true.txt          
  inflating: __MACOSX/texts/._true.txt  
  inflating: texts/kouros.txt        
  inflating: __MACOSX/texts/._kouros.txt  
  inflating: texts/medici.txt        
  inflating: __MACOSX/texts/._medici.txt  


In [10]:
import os
import pandas as pd

def read_files_to_dataframe(folder_path: str) -> pd.DataFrame:
    # List all text files in the folder
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

    # Read each file's content into a list
    data = []
    for file_name in file_names:
        with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
            content = file.read()
            data.append(content)

    # Create a DataFrame with the file contents
    df = pd.DataFrame(data, columns=['content'])

    return df

# Usage example
folder_path = 'texts'
df = read_files_to_dataframe(folder_path)
print(df)

                                             content
0  Marion True was Curator of Antiquities at the ...
1  The Getty kouros (youth) is a 2 m high marble ...
2  Giacomo Medici is an Italian antiquities deale...


In [32]:
import importlib
import sys
importlib.reload(sys.modules['prompt'])
from prompt import DEFAULT_BASE_PROMPT, create_prompt

model_name = "gpt-4o"
temperature = 0
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name=model_name, temperature=temperature)
parser = PydanticOutputParser(pydantic_object=CulturalHeritageSchema)
prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df["content"][0])
prompt


[HumanMessage(content='\nExtract information regarding cultural heritage crime from the following text. \n\nIn particular, please provide the following information:\n- A list of entities involved in the text, including their types and detailed descriptions.\n- A list of relations that define how these entities interact with one another.\n- A set of patterns that describe specific relational configurations between these entities, providing detailed contextual explanations.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Entity": {"description": "Entity schema", "propert

In [33]:
example = await run(llm, prompt, parser)
example

CulturalHeritageSchema(entities=[Entity(name='Marion True', description='actor'), Entity(name='J. Paul Getty Museum', description='museum'), Entity(name='Jiri Frel', description='actor'), Entity(name='Getty Aphrodite', description='artefact'), Entity(name='fourth-century BC gold funerary wreath', description='artefact'), Entity(name='Barbara and Lawrence Fleischman Collection', description='collection'), Entity(name='Kanakaria mosaics', description='artefact'), Entity(name='Sevso Treasure', description='artefact'), Entity(name='Francavilla Maritima', description='place'), Entity(name='Getty Villa', description='place'), Entity(name='Christos Michaelides', description='actor'), Entity(name='Robin Symes', description='dealer'), Entity(name='Robert Hecht', description='dealer'), Entity(name='Giacomo Medici', description='dealer'), Entity(name='Guglielmo Collection', description='collection'), Entity(name='Barbara Fleischman', description='actor'), Entity(name='Greek prosecutors', descript

In [ ]:
## on all the text
raw_outputs = []
outputs = []

for i in range(df.shape[0]):

    prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df["content"][i])
    result = await run(llm, prompt, parser)
    raw_outputs.append(result)

    output_map = {
        "article_id":i,
    }

    if isinstance(result, CulturalHeritageSchema):
        output =  output_map | result.dict()
        outputs.append(output)
    else:
        output_map["raw_llm_output"] = result
        outputs.append(output_map)

    time.sleep(1)